In [1]:

# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

In [2]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(27, 'documents')
(9, 'classes', [u'courses', u'demo', u'goodbye', u'greeting', u'hours', u'opentoday', u'payments', u'thanks', u'today'])
(50, 'unique stemmed words', [u"'d", u"'s", u'a', u'acceiv', u'anyon', u'ar', u'bye', u'can', u'card', u'cash', u'cheaqu', u'class', u'cours', u'credit', u'day', u'demo', u'do', u'good', u'goodby', u'hav', u'hello', u'help', u'hi', u'hour', u'how', u'i', u'is', u'kind', u'lat', u'lik', u'of', u'on', u'op', u'schedule', u'see', u'tak', u'teach', u'thank', u'that', u'the', u'ther', u'tim', u'to', u'today', u'we', u'what', u'when', u'which', u'yo', u'you'])


In [4]:
#nltk.download('punkt')


In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])


In [6]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 1.16018 | time: 0.012s
| Adam | epoch: 1000 | loss: 1.16018 - acc: 0.9212 -- iter: 24/27
Training Step: 4000  | total loss: 1.04632 | time: 0.016s
| Adam | epoch: 1000 | loss: 1.04632 - acc: 0.9291 -- iter: 27/27
--
INFO:tensorflow:/Users/ankitakash2007/projects/Untitled Folder/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [7]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )
